## Data Curation Using the Relational Model

---
**Important**

* The ```#grade``` tag in notebook cells is required for grading purposes. Do not remove.*
* Remember to save your notebook before submitting.
---

Data curation is concerned with all aspects of data management required to efficiently and reliably support analysis and reuse. This includes everything from initial data acquisition to organization, integration, access, and preservation. Data typically originate from a variety of sources, in a variety of formats, and may need transformation. Curation often involves the selection of a basic model for storage and organization that requires an understanding of the data as well as any objectives for its use.

In this exercise, we explore the application of the widely-used relational model to support analysis as well as how it relates to Wickham's concept of <i>tidy data</i>. It is assumed that you have a basic understanding of relational databases from a prior database systems course including schema design, constraints, and queries.

### Learning Objectives

* Apply strategies for data curation using the relational model including implementing a relational schema, integrity constraints, and queries.
* Gain experience analyzing characteristics of a dataset.
* Gain experience with principles of tidy data.

### Deliverables:
* `schema.sql`: File defining database schema.
* `Workbook.ipynb`: This notebook with graded functions implemented.

### Scenario

You are part of a team tasked with exploring factors involved in food safety violations. As a starting point, the team has decided to work with the publicly-available [Food Inspections](https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5) data from the City of Chicago. 

When curating an existing dataset, we have the option to leave it as-is or to transform it into another model to facilitate use. In this exercise, you will create a simple relational database that can be used for subsequent analysis. 

Using a subset of the source data provided in `data/restaurants-subset.csv` you will:
* Implement a relational database schema that can be used to address pre-defined questions.
* Implement queries to return specific values.
* Implement queries to return analysis-ready data in tidy format.

**You are restricted to using the [DuckDB Python API](https://duckdb.org/docs/api/python/overview).** While there are many ways to answer these question using the data, the purpose of this assignment is to use the relational model.

At the end of the assignment, your schema and queries, implemented as Python functions in this notebook, will be used to answer the following questions:
* What is the total number of inspections?
* What is the total number of violations?
* What are the *n* most common violations?
* What are the *n* restaurants with the most violations?

In [1]:
import duckdb

### 1. Inspect the Data

Read the [About](https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5/about_data) page on the City of Chicago Data Portal. Use the Data Portal, this workspace environment, and/or `duckdb` API to inspect the dataset focusing on structure and attributes.

For example, you can use DuckDB's built-in ability to query the CSV data directly and return a dataframe:

In [2]:
duckdb.sql("SELECT * FROM 'data/restaurants-subset.csv' limit 5").df()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2222594,LA ESPERANZA,LA ESPERANZA,2180050.0,Grocery Store,Risk 1 (High),3000 S DRAKE AVE,CHICAGO,IL,60623.0,2018-09-19,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.838816,-87.713565,"(41.83881606238869, -87.71356485651883)"
1,2553999,BIRRIERIA JALISCO,BIRRIERIA JALISCO,1772607.0,Restaurant,Risk 1 (High),2462 W 47TH ST,CHICAGO,IL,60632.0,2022-04-07,Canvass Re-Inspection,Pass,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...,41.808511,-87.686695,"(41.808511205858146, -87.68669535004372)"
2,2232545,LITTLE KIDS VILLAGE LEARNING,LITTLE KIDS VILLAGE LEARNING,2215838.0,Daycare (2 - 6 Years),Risk 1 (High),2656 W 71ST ST,CHICAGO,IL,60629.0,2018-10-30,Canvass,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.764689,-87.690441,"(41.764689317705226, -87.6904408401805)"
3,2594512,HEARTY CAFE PANCAKE HOUSE,HEARTY CAFE PANCAKE HOUSE,2817504.0,Restaurant,Risk 1 (High),9623 S WESTERN AVE,CHICAGO,IL,60643.0,2024-05-14,Canvass Re-Inspection,Pass,41. WIPING CLOTHS: PROPERLY USED & STORED - Co...,41.718398,-87.681838,"(41.71839819308241, -87.68183836584416)"
4,2570937,Burley Elementary,Burley Elementary,22421.0,School,Risk 1 (High),1630 W Barry (3100N),CHICAGO,IL,60657.0,2023-02-08,Canvass Re-Inspection,Pass,None,41.937965,-87.669852,"(41.93796493348329, -87.66985204161114)"


### 2. Create Your Schema
* Create a file named `schema.sql` that can be used to create your database. Note that any database files (e.g., `.db`) will be ignored during grading.
* Your schema must contain a table named `INSPECTIONS` that stores the inspection data. This table must have a primary key.
* During autograding, we will use code similar to the following to create and populate the database.

In [4]:
with duckdb.connect("inspections.db") as con:
    with open("schema.sql", "r") as f:
        # Create the DB schema
        con.sql(f.read())
        # Import data
        con.sql("COPY inspections from 'data/restaurants-subset.csv'")

Implement a function `count_inspections()` that uses `duckdb.sql()` to query the database and return the total number of inspections.

In [5]:
#grade
def count_inspections(con):
    result = con.sql("SELECT COUNT(*) FROM INSPECTIONS").fetchone()[0]

    return result

### 3. Transform the Data

As you may have noticed, the `Violations` field contains nested records for each of the violations found during a single inspection. Note that the violations refer to a list of standard [codes and descriptions](https://www.chicago.gov/city/en/depts/cdph/provdrs/food_safety/svcs/understand_healthcoderequirementsforfoodestablishments.html) but also contain information specific to the inspection.

Update your `schema.sql` to create additional tables to store the violations data.

* `VIOLATIONS`: Table to store unique information about violations. It must have a primary key that serves as a foreign key for the `INSPECTION_VIOLATIONS` table.
* `INSPECTION_VIOLATIONS`: Table to store information about violations associated with each inspection. It must have a primary key and foreign keys to `INSPECTIONS` and `VIOLATIONS`.

Write a function `parse_violations()` that uses the DuckDB API to parse the `INSPECTIONS.Violations` field and populate the two new tables.

In [12]:
#grade 
def parse_violations(con):
    import pandas as pd
    
    # Good test candidate:  limit 1 offset 102
    df = con.sql("SELECT \"Inspection ID\", Violations FROM inspections where Violations not null").df()
    
    violations = {}
    inspection_violations = {}
    inspection_violation_primary_id = 1
    
    for index, row in df.iterrows():
        inspection_id = row["Inspection ID"]
        curr_violations = row["Violations"]
        if curr_violations:
            violation_list = curr_violations.split("|")

            for entry in violation_list:
                vio_split = entry.strip().split(" - ", 1)

                violation = vio_split[0].strip()
                violation_comments = ""
                
                if(len(vio_split) > 1):
                    violation_comments = entry.strip().split(" - ", 1)[1].strip()
                
                code = violation.split(". ", 1)[0]
                code = code.strip()
                description = violation.split(". ", 1)[1].strip()
                
                if code not in violations:
                    violations[code] = {
                        "code": code,
                        "description": description,
                    }
                
                inspection_details_dirty = []
                if len(violation_comments) > 1:
                    inspection_details_dirty = violation_comments.split("Comments:", 1)
                
                inspection_details = ""
                if len(inspection_details_dirty) > 1:
                    inspection_details = inspection_details_dirty[1].strip()
                
                key = str(inspection_id) + ":" + code + ":" + inspection_details

                if key not in inspection_violations:
                    inspection_violations[key] = {
                        "inspection_violation_id": inspection_violation_primary_id,
                        "inspection_id": inspection_id,
                        "violation_id": code,
                        "inspection_details": inspection_details
                    }
                    inspection_violation_primary_id += 1
                    
    inspection_violations_df = pd.DataFrame([
        {"inspection_violation_id": iv["inspection_violation_id"], "inspection_id": iv["inspection_id"], "violation_id": iv["violation_id"], "inspection_details": iv["inspection_details"]}
        for key, iv in inspection_violations.items()
    ])
    print(inspection_violations_df)
    
    violations_df = pd.DataFrame([
        {"code": v["code"], "description": v["description"]}
        for code, v in violations.items()
    ])
    
    con.sql("INSERT INTO violations SELECT * FROM violations_df")
    con.sql("INSERT INTO inspection_violations SELECT * FROM inspection_violations_df")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

0    1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...
1    37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...
2    47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...
3    41. WIPING CLOTHS: PROPERLY USED & STORED - Co...
4                                                 None
Name: Violations, dtype: object
1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLEDGE, AND PERFORMS DUTIES - Comments: 2-102.11 PIC DIDN'T DEMONSTRATE KNOWLEDGE OF FOOD OPERATIONS.MUST PROVIDE.(PRIORITY FOUNDATION-NO CITATION ISSUED) | 3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING - Comments: 2-102.14(N)OBSERVED NO EMPLOYEES HEALTH POLICY PROCEDURE.MUST PROVIDE AND MAINTAIN.(PRIORITY FOUNDATION-NO CITATION ISSUED) | 5. PROCEDURES FOR RESPONDING TO VOMITING AND DIARRHEAL EVENTS - Comments: 2-501.11 OBSERVED NO CLEAN-UP POLICY PROCEDURE FOR VOMITING AND DIARRHEA.MUST PROVIDE AND MAINTAIN.(PRIORITY FOUNDATION- NO CITATION ISSUED) | 23. PROPER DATE MARKING AND DISPOSITION

### 4. Queries

Write a function `get_top_violations(n)` that queries the database and returns the top `n` violations based on a count of inspection/violation pairs:

In [1]:
#grade
def get_top_violations(con, n):
    """
    Use con.sql().df() to execute a query to return the top n violations.
    Return a dataframe with violation_id and count sorted descending.
    """
    return con.sql("SELECT violation_id, COUNT(*) AS count FROM inspection_violations GROUP BY violation_id ORDER BY count DESC LIMIT " + str(n)).df()

Write a function `get_restaurants(n)` that queries the database and returns the `n` restaurants with the most violations.

In [ ]:
#grade
def get_top_restaurants(con, n):
    """
    Use con.sql().df() to execute a query to return the top n restaurants with the most violations.
    Return a dataframe with columns dba_name, address, count sorted by count descending.
    """
    
    string_q = """
        SELECT
            ins."DBA Name" AS DBA_NAME,
            ins.Address AS ADDRESS,
            COUNT(v.violation_id) AS COUNT
        FROM inspections AS ins JOIN inspection_violations AS v ON ins."Inspection ID" = v.inspection_id
        WHERE ins."Facility Type" = 'Restaurant'
        GROUP BY DBA_NAME, ADDRESS ORDER BY COUNT DESC, DBA_NAME ASC, ADDRESS ASC LIMIT 
    """
    result = con.sql(string_q + str(n)).df()
    return result

### 5. Tidy Data
Recall Wickham's *tidy data* principles:
* Each variable forms a column
* Each observation forms a row
* Each type of observational unit forms a table

Write a function ```get_tidy_violations()``` that returns a tidy dataframe where each observation represents an inspection violation. Your dataframe must contain the inspection ID, inspection date, violation ID and inspection result.

In [ ]:
#grade
def get_tidy_violations(con):
    """
    Use con.sql().df() to execute a query to return a tidy dataframe of violations where each inspection violation is an observation.
    Return a dataframe with columns inspection_id, inspection_date, violation_id, and results.
    """
    string = """
        SELECT 
            ins."Inspection ID" AS inspection_id,
            ins."Inspection Date" AS inspection_date,
            iv.violation_id,
            ins."Results" AS results
        FROM inspections ins
        JOIN inspection_violations iv ON ins."Inspection ID" = iv.inspection_id
        ORDER BY inspection_id ASC
        """
    return con.execute(string).df()

Once you have completed the assignment, select **Submit & Grade** on the PrairieLearn assignment page.